In [1]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import random
from readAndSortCsv import read_and_sort_csv
from TradingStatisticsCalculator import TradingStatisticsCalculator
from HybridCNN import HybridCNN
from ta import trend, momentum, volatility, volume

In [2]:
MODEL_PATH = 'simple1dcnn_state_dict.pth'
required_columns = ['date', 'open', 'high', 'low', 'close', 'volume']
file_path = r"DataFromBinance.csv"
input_window = 150
output_window = 15
np.set_printoptions(formatter={'float_kind': lambda x: f'{x:.2f}'})
random.seed(42)
np.random.seed(42)

In [3]:
algo_calc = TradingStatisticsCalculator(
    initial_capital=5000.0,
    position_size_dollars=1000.0,
    close_idx=3,
    high_idx=1,
    low_idx=2,
    commission_rate=0.0005,
    tp_percent=0.0034,
    sl_percent=0.0033
)

In [4]:
df_raw = read_and_sort_csv(file_path, required_columns)[-30000:]
df_raw.sort_index(ascending=True, inplace=True)
df_raw

,open,high,low,close,volume
date,,,,,
2024-12-26 00:00:00,99429.61,99536.00,99429.60,99515.97,33.08546
2024-12-26 00:01:00,99515.97,99572.01,99500.00,99557.60,48.33172
2024-12-26 00:02:00,99557.61,99580.00,99520.15,99580.00,8.74854
2024-12-26 00:03:00,99580.00,99762.50,99579.99,99748.97,96.45024
2024-12-26 00:04:00,99748.97,99800.00,99713.55,99792.02,28.84950
...,...,...,...,...,...
2024-12-29 23:56:00,93784.03,93807.43,93750.64,93807.43,20.41480
2024-12-29 23:57:00,93807.43,93807.43,93733.69,93749.11,8.99659
2024-12-29 23:58:00,93749.11,93749.11,93692.00,93692.31,10.27729


In [5]:
def compute_features(df_input):
    df = df_input.iloc[:]
    df = df.iloc[::-1]
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['EMA_20'] = df['close'].ewm(span=20, adjust=False).mean()

    df['RSI_14'] = momentum.RSIIndicator(close=df['close'], window=14).rsi()

    macd = trend.MACD(close=df['close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Diff'] = macd.macd_diff()

    bollinger = volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
    df['Bollinger_High'] = bollinger.bollinger_hband()
    df['Bollinger_Low'] = bollinger.bollinger_lband()
    df['Bollinger_Middle'] = bollinger.bollinger_mavg()

    df['ATR_14'] = volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14).average_true_range()

    df['OBV'] = volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()

    stochastic = momentum.StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14, smooth_window=3)
    df['Stochastic_%K'] = stochastic.stoch()
    df['Stochastic_%D'] = stochastic.stoch_signal()

    ichimoku = trend.IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
    df['Ichimoku_A'] = ichimoku.ichimoku_a()
    df['Ichimoku_B'] = ichimoku.ichimoku_b()
    df['Ichimoku_Base_Line'] = ichimoku.ichimoku_base_line()
    df['Ichimoku_Conversion_Line'] = ichimoku.ichimoku_conversion_line()
    df.dropna(inplace=True)

    return df.iloc[::-1]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HybridCNN(num_features=22, seq_len=150, num_classes=3)

model.load_state_dict(torch.load(r"C:\GitCnn\CnnTrading\CnnTrans\HybridCNN_best_on_valid_state_dict.pth", map_location=device))

model.to(device)


C:\Users\Dima03\AppData\Local\Temp\ipykernel_7904\3530465443.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\GitCnn\CnnTrading\CnnT

HybridCNN(
  (shared_conv1): Conv1d(22, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (shared_relu): ReLU()
  (shared_dropout): Dropout1d(p=0.4, inplace=False)
  (feature_cnns): ModuleList(
    (0-21): 22 x Sequential(
      (0): Conv1d(1, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): Dropout1d(p=0.6, inplace=False)
      (3): Conv1d(128, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): ReLU()
      (5): Dropout1d(p=0.4, inplace=False)
    )
  )
  (fc1): Linear(in_features=110400, out_features=256, bias=True)
  (fc_dropout): Dropout(p=0.8, inplace=False)
  (fc2): Linear(in_features=256, out_features=3, bias=True)
)

In [7]:
class Predictor:
    def __init__(
        self,
        model):
        self.model = model
        self.model.eval()
    def predict(self, df_input):
        X_single_scaled = MinMaxScaler().fit_transform(df_input)  # shape (600,5)
        X_single_scaled = np.expand_dims(X_single_scaled, axis=0)  # => (1,600,5)
        X_single_transposed = np.transpose(X_single_scaled, (0, 2, 1))  # => (1,5,600)
        X_single_tensor = torch.from_numpy(X_single_transposed).float().to(device)
        with torch.no_grad():
            output = model(X_single_tensor)   # => shape (1,3)
            _, predicted = torch.max(output, 1)
            predicted_label = predicted.item()  # 0,1,2

        label_map = {0:"short",1:"flat",2:"long"}
        return label_map[predicted_label]

In [8]:
predictor = Predictor(model)

In [9]:
class InputProvider:
    def __init__(self, window_size=183):
        self.window = window_size
        self.current_index = 0
        self.next_input = None

    def can_get_next_input(self):
        if self.current_index + self.window > len(df_raw):
            return False

        df_slice = df_raw.iloc[self.current_index : self.current_index + self.window]
        self.next_input = compute_features(df_slice)
        self.current_index += 1
        return True

    def get_next_input(self):
        result = self.next_input.to_numpy()
        return result


In [10]:
input_provider = InputProvider()

In [11]:
while input_provider.can_get_next_input():
    df_input = input_provider.get_next_input()
    predicted_label = predictor.predict(df_input)
    algo_calc.process_candle(df_input[0], predicted_label)

In [12]:
algo_stats = algo_calc.get_statistics()
algo_stats.to_dataframe().T

,0
Initial Capital,5000.000000
Final Capital,4905.425073
Total Profit,-94.574927
Average Profit,-1.062640
Return on Investment (ROI),-1.891499
Number of Trades,89.000000
Long Trades,47.000000
Short Trades,42.000000
Flat Trades,161.000000
Position Size per Trade,1000.000000
